## Imports and Hyperparameters

In [ ]:
# TODO

In [ ]:
# TODO

## Load Data and Original Labels

In [ ]:
def load_dataset_from_folder(base_path, max_data_length):
    """
    Load time-series CSV files organized in subfolders per class.

    Parameters
    - base_path: str or Path to the folder containing one subfolder per class.
                 Each class subfolder should contain CSV files (all same shape).
    - max_data_length: int, maximum length of the time series data (number of timesteps).

    Returns
    - X: numpy array of shape (n_samples, n_variables, n_timesteps)
    - y: numpy array of integer labels (0..n_classes-1)

    Prints shapes of X and y.
    """

    data = []
    labels = []

    # Find subfolders (classes) in deterministic order
    class_folders = sorted(
        [
            d for d in os.listdir(base_path)
            if os.path.isdir(os.path.join(base_path, d))
        ]
    )
    if not class_folders:
        raise ValueError(f"No class subfolders found in {base_path}")

    # Build list of files to load (with labels) to show a single progress bar
    files_to_load = []
    for label, class_name in enumerate(class_folders):
        class_dir = os.path.join(base_path, class_name)
        # Collect CSV files in deterministic order
        csv_files = sorted(
            [
                f for f in os.listdir(class_dir)
                if os.path.isfile(os.path.join(class_dir, f)) and f.lower().endswith(".csv")
            ]
        )
        if not csv_files:
            raise ValueError(f"No CSV files found in class folder {class_dir}")

        for fname in csv_files:
            files_to_load.append((os.path.join(class_dir, fname), label))

    # Iterate with tqdm progress bar
    for csv_path, label in tqdm(files_to_load, desc="Loading dataset", unit="file"):
        df = pd.read_csv(csv_path, index_col=0)
        arr = df.values.T  # transpose to match original layout: (variables, timesteps)
        data.append(arr)
        labels.append(label)

    # Ensure all samples have the same shape
    shapes = {a.shape for a in data}
    if len(shapes) != 1:
        raise ValueError(f"Inconsistent sample shapes found: {shapes}")

    X = np.array(data)
    y = np.array(labels)

    # Resize data to max_data_length if necessary
    if X.shape[2] > max_data_length:
        X = X[:, :, :max_data_length]

    print("Data shape: {}".format(X.shape))
    print("Labels shape: {}".format(y.shape))

    return X, y

## Experiments

### Tennessee-Eastman Process (TEP)

In [ ]:
X, y = load_dataset_from_folder("data/tep", MAX_DATA_LENGTH)

### Fluidized Catalytic Cracking (FCC)

In [ ]:
X, y = load_dataset_from_folder("data/fcc", MAX_DATA_LENGTH)